In [3]:
import json
with open("data/extracted_clauses/extracted_clauses.json", "r") as f:
    clauses_parents = json.load(f)

In [1]:
# load fine-tuned model

from transformers import BertTokenizer, BertForSequenceClassification

# Define the path where the fine-tuned model is saved
model_save_path = "/code/Python/situation_entity_type_classification/trained_model"

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained(model_save_path)

# Load the fine-tuned model
model = BertForSequenceClassification.from_pretrained(model_save_path)


/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch
from tqdm import tqdm  # Import tqdm for the progress bar

# Ensure the model is on the correct device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

def classify_clauses(clauses, model, tokenizer, device):
    results = []
    # Wrap the loop with tqdm for a progress bar
    for clause in tqdm(clauses, desc="Classifying clauses"):
        inputs = tokenizer(clause['sentence'], return_tensors='pt', padding=True, truncation=True, max_length=128)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=1)
            clause['category'] = predictions.item()
        results.append(clause)
    return results

# Classify clauses
annotated_clauses = classify_clauses(clauses_parents, model, tokenizer, device)

Classifying clauses:   1%|          | 8135/1468738 [38:19<114:41:37,  3.54it/s]   


KeyboardInterrupt: 

In [8]:
import json

# Save annotated clauses to a JSON file
with open('annotated_clauses.json', 'w') as f:
    json.dump(annotated_clauses, f, indent=4)


In [11]:
import nltk

# Download the necessary NLTK resources
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sunnyyu/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True